## NPX - spont

spikeinterf0_100_5

In [1]:
# SETUP PACKAGES 
%load_ext autoreload
%autoreload 2
import os
import spikeinterface as si
import numpy as np
import warnings
from pynwb.file import NWBFile, Subject
from pynwb import NWBHDF5IO
from datetime import datetime
from pathlib import Path
from pynwb import NWBHDF5IO
import uuid
from datetime import datetime
from dateutil.tz import tzlocal
import spikeinterface.extractors as se
from neuroconv.tools.spikeinterface import add_recording, add_sorting

warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 


# SET PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/preprint_2023"
os.chdir(PROJ_PATH)
from src.nodes.utils import get_config
from src.nodes import utils
from src.nodes.dataeng.silico import probe_wiring

# SETUP CONFIG
data_conf, param_conf = get_config("silico_neuropixels", "concatenated").values()
REC_PATH = data_conf["probe_wiring"]["full"]["output"]
GT_PATH = data_conf["sorting"]["simulation"]["ground_truth"]["output"]
NWB_PATH = data_conf["nwb"]

# SET PARAMETERS
subject_id = "001"
nwbfile_path = Path.cwd() / "spikeinterface_to_nwb.nwb"

2024-11-06 11:11:38,412 - root - utils.py - get_config - INFO - Reading experiment config.
2024-11-06 11:11:38,480 - root - utils.py - get_config - INFO - Reading experiment config. - done


In [2]:
def init_nwbfile(
    subject_id="001",
    session_description="Biophysical simulation in the spontaneous regime",
    identifier=str(uuid.uuid4()),
    session_start_time=datetime.now(tzlocal()),
    experimenter="Steeve Laquitaine",
    lab="Blue Brain Project",
    institution="EPFL",
    experiment_description="Biophysical simulation",
    session_id="001",
    related_publications="doi:",
):

    # parmaetrize session file
    nwbfile = NWBFile(
        session_description=session_description,
        identifier=identifier,
        session_start_time=session_start_time,
        experimenter=experimenter,
        lab=lab,
        institution=institution,
        experiment_description=experiment_description,
        session_id=session_id,
        related_publications=related_publications,
    )

    # subject metadata
    nwbfile.subject = Subject(subject_id=subject_id, species="Rat", age="P14D")
    return nwbfile


def create_dataset():
    from spikeinterface.core import generate_ground_truth_recording

    recording, sorting = generate_ground_truth_recording(
        num_channels=8,
        num_units=3,
        durations=[60 * 10.0],
        seed=0,
    )

    recording = recording.rename_channels(
        new_channel_ids=["A", "B", "C", "D", "E", "F", "G", "H"]
    )
    sorting = sorting.rename_units(new_unit_ids=["Unit A", "Unit B", "Unit C"])

    # Add new properties to the recording and sorting
    recording.set_property(
        key="a_channel_property",
        values=[f"property {channel}" for channel in recording.get_channel_ids()],
    )
    recording.set_property(
        key="brain_area",
        values=[f"Area {channel}" for channel in recording.get_channel_ids()],
    )
    sorting.set_property(
        key="a_unit_property",
        values=["property Unit A", "property Unit B", "property Unit C"],
    )
    sorting.delete_property("gt_unit_locations")
    return recording, sorting

In [3]:
# get raw wired RecordingExtractor
Recording = si.load_extractor(REC_PATH)
Recording = probe_wiring.run(Recording, data_conf, param_conf)

# get SortingExtractor
# remove features that are not handeled because 3D
Sorting = si.load_extractor(GT_PATH)
Sorting.delete_property("orientation")

In [ ]:
# create NWB file and add extractors
nwbfile = init_nwbfile()
add_recording(recording=Recording, nwbfile=nwbfile)
add_sorting(sorting=Sorting, nwbfile=nwbfile)

# save this to a NWB file
utils.create_if_not_exists(os.path.dirname(NWB_PATH))
with NWBHDF5IO(path=NWB_PATH, mode="w") as io:
    io.write(nwbfile)

2024-11-05 21:48:21,281 - root - utils.py - create_if_not_exists - INFO - The following path has been created /gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/0_silico/neuropixels/concatenated_campaigns/nwb


In [6]:
# load NWB file to check
Recording = se.NwbRecordingExtractor(NWB_PATH)
Sorting = se.NwbSortingExtractor(NWB_PATH)